In [1]:
import urllib.request
import json
import datetime
import time
import re
import pandas as pd

country_code = 'vn'


def get_json_product(itemid, limit, offset, shopid, type=0):
    '''Get JSON of a product
    * type = 0: get all ratings
    * type = 1..5: get ratings based on rating stars
    * country_code = 'vn' or 'sg'
    '''
    url = 'https://shopee.{}/api/v2/item/get_ratings?filter=0&flag=1&itemid={}&limit={}&offset={}&shopid={}&type={}'.format(
        country_code, itemid, limit, offset, shopid, type)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data


def get_json_recommend(limit, offset):
    url = 'https://shopee.{}/api/v4/recommend/recommend?bundle=daily_discover_main&limit={}&offset={}'.format(
        country_code, limit, offset)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data


def get_json_campaign(label, limit, offset):
    url = 'https://shopee.{}/api/v4/recommend/recommend?bundle=daily_discover_campaign&label={}&limit={}&offset={}'.format(
        country_code, label, limit, offset)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data


def remove_adjacent_duplicates(str):
    return re.sub(r'(.)\1+', r'\1\1', str)


def format_string(str):
    if str:
        locale_chars = ''
        if country_code == 'vn':
            locale_chars = ' ,.\n\tABCDEGHIKLMNOPQRSTUVXYabcdeghiklmnopqrstuvxyÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
        elif country_code == 'sg':
            locale_chars = ' ,.\n\tABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
        bad_chars = [('\t', ', '), ('\n', '. '), ('  ', ' '), (' .', '.'),
                     (' ,', ','), ('..', '.'), (',,', ','), (',.', '.'), ('.,', ',')]
        # Keep only specific characters
        str = ''.join(c for c in str if c in locale_chars)
        str = remove_adjacent_duplicates(str)
        for c in bad_chars:
            str = str.replace(c[0], c[1])
        str = str.strip()
    return str


def get_ratings_from_json(json_data, min_len_str=4):
    data = json_data['data']
    ratings = data['ratings'] if data != None else None
    result = []
    if ratings != None:
        for r in ratings:
            itemid = r['itemid']
            shopid = r['shopid']
            userid = r['userid']
            cmtid = r['cmtid']
            mtime = datetime.datetime.fromtimestamp(
                r['mtime']).strftime('%d-%m-%Y %H:%M:%S')
            rating_star = r['rating_star']
            comment = format_string(r['comment'])
            if comment != None and len(comment) >= min_len_str:
                result.append(
                    {
                        'itemid': itemid,
                        'shopid': shopid,
                        'userid': userid,
                        'cmtid': cmtid,
                        'mtime': mtime,
                        'rating_star': rating_star,
                        'comment': comment
                    })
    return result


def get_products_from_json(json_data, get_top_product=False):
    data = json_data['data']
    sections = data['sections'] if data != None else []
    result = []
    for s in sections:
        data = s['data']
        item = data['item']
        if item != None:
            for i in item:
                shopid = i['shopid']
                itemid = i['itemid']
                result.append(
                    {
                        'shopid': shopid,
                        'itemid': itemid
                    })
        if get_top_product:
            top_product = data['top_product']
            if top_product != None:
                for t in top_product:
                    list = t['list']
                    data = list['data']
                    item_lite = data['item_lite']
                    if item_lite != None:
                        for i in item_lite:
                            shopid = i['shopid']
                            itemid = i['itemid']
                            result.append(
                                {
                                    'shopid': shopid,
                                    'itemid': itemid
                                })
    return result


def get_all_ratings(itemid, shopid, limit=6, offset=0, min_len_cmt=4, type=0):
    result = []
    while True:
        json_data = get_json_product(itemid, limit, offset, shopid, type)
        ratings = get_ratings_from_json(json_data, min_len_cmt)
        if ratings == []:
            break
        else:
            result += ratings
        offset += limit
    return result


def get_all_recommended_products(max_products=100, limit=10, offset=0, get_top_product=False):
    result = []
    if max_products < limit:
        limit = max_products
    while True:
        start_time = time.time()
        # Notes: The number of products may be smaller than limit number although max_products < limit
        # So the number of result can be larger than the max_products
        json_data = get_json_recommend(limit, offset)
        products = get_products_from_json(json_data, get_top_product)
        if products == [] or len(result) >= max_products:
            break
        else:
            result += products
            print('Đã lấy về {} sản phẩm trên tổng số tối đa {} sản phẩm. Mất {:0.2f} mili giây'.format(
                len(result), max_products, (time.time() - start_time)*1000))
        offset += limit
    return result


def get_all_campaign_products(label, max_products=100, limit=10, offset=0):
    result = []
    if max_products < limit:
        limit = max_products
    while True:
        start_time = time.time()
        # Notes: The number of products may be smaller than limit number although max_products < limit
        # So the number of result can be larger than the max_products
        json_data = get_json_campaign(label, limit, offset)
        products = get_products_from_json(json_data, False)
        if products == [] or len(result) >= max_products:
            break
        else:
            result += products
            print('Đã lấy về {} sản phẩm trên tổng số {} sản phẩm, tối đa {} sản phẩm. Mất {:0.2f} mili giây'.format(
                len(products), len(result), max_products, (time.time() - start_time)*1000))
        offset += limit
    return result


def export_to_text_file(array_of_json, filename, only_header=False):
    f = open(filename, 'a+', encoding='utf-8')
    if only_header:
        f.write('userid\tcmtid\tmtime\trating_star\tcomment\n')
    else:
        for j in array_of_json:
            f.write('{}\t{}\t{}\t{}\t{}\n'.format(
                j['userid'], j['cmtid'], j['mtime'], j['rating_star'], j['comment']))
    f.close()
    
def export_to_csv(array_of_json, filename, only_header=False):
    df = pd.DataFrame(array_of_json)
    if only_header:
        df.to_csv(filename, header=True, index=False, mode='a', encoding='utf-8')
    else:
        df.to_csv(filename, header=False, index=False, mode='a', encoding='utf-8')


def collect_reviews_product(filename, max_products, min_len_cmt=4, types=[0]):
    '''Collect all reviews of products with specific rating_star
    * type = array [0]: get all rating_stars
    * type = array [1..5]: get only these rating_stars
    '''
    products = get_all_recommended_products(
        max_products=max_products, get_top_product=True)
    # products = get_all_campaign_products(1005922, max_products)
    length_products = len(products)
    export_to_text_file(None, filename, True)
    for p in products:
        start_time = time.time()
        itemid = p['itemid']
        shopid = p['shopid']
        ratings = []
        if types != None and types != []:
            for t in types:
                ratings += get_all_ratings(
                    itemid, shopid, min_len_cmt=min_len_cmt, type=t)
        else:
            ratings += get_all_ratings(itemid, shopid, min_len_cmt=min_len_cmt)
        
        export_to_csv(ratings, filename)
        length_products -= 1
        print('Đã thu thập và ghi {} đánh giá của sản phẩm {} tại shop {}. Còn {} sản phẩm nữa. Mất {:0.2f} mili giây'.format(
            len(ratings), itemid, shopid, length_products, (time.time() - start_time)*1000))


def remove_duplicate_column(filename, col_check):
    df = pd.read_csv(filename, delimiter='\t')
    print(df['rating_star'].value_counts().sort_index(ascending=True))
    df.drop_duplicates(col_check, inplace=True)
    print(df['rating_star'].value_counts().sort_index(ascending=True))
    df.to_csv(filename, sep='\t', index=False)


def prune(filename):
    df = pd.read_csv(filename, delimiter='\t')
    min = df.groupby('rating_star').agg('count')['comment'].min()
    for i in [1, 2, 3, 4, 5]:
        rows = df.loc[df['rating_star'] == i]
        rows = rows.sort_values(
            by='comment', key=lambda x: x.str.len(), ascending=False)
        rows = rows.head(min)
        header = True if i == 1 else False
        rows.to_csv('pruned_' + filename, mode='a',
                    index=False, sep='\t', header=header)

In [2]:
collect_reviews_product("reviews_45s.csv", max_products=500, min_len_cmt=4, types=[4, 5])

Đã lấy về 8 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 466.95 mili giây
Đã lấy về 16 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 628.40 mili giây
Đã lấy về 22 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 698.42 mili giây
Đã lấy về 29 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 612.15 mili giây
Đã lấy về 36 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 599.39 mili giây
Đã lấy về 41 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 670.36 mili giây
Đã lấy về 46 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 434.62 mili giây
Đã lấy về 51 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 580.52 mili giây
Đã lấy về 56 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 588.83 mili giây
Đã lấy về 61 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 586.64 mili giây
Đã lấy về 66 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 494.94 mili giây
Đã lấy về 71 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 723.99 mili giây
Đã lấy về 81 sản phẩm trên tổng số tối đa 500 sản phẩm. Mất 617.41 mili giây


Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 467 sản phẩm nữa. Mất 3618.94 mili giây
Đã thu thập và ghi 445 đánh giá của sản phẩm 21617915490 tại shop 788908334. Còn 466 sản phẩm nữa. Mất 22263.25 mili giây
Đã thu thập và ghi 6 đánh giá của sản phẩm 25400385770 tại shop 677686374. Còn 465 sản phẩm nữa. Mất 570.26 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 19995035473 tại shop 678798230. Còn 464 sản phẩm nữa. Mất 585.88 mili giây
Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 463 sản phẩm nữa. Mất 201709.26 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 462 sản phẩm nữa. Mất 3109.56 mili giây
Đã thu thập và ghi 7 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 461 sản phẩm nữa. Mất 2656.38 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 460 sản phẩm nữa. Mất 6069.43 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 

Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 398 sản phẩm nữa. Mất 189953.93 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 397 sản phẩm nữa. Mất 1179.23 mili giây
Đã thu thập và ghi 7 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 396 sản phẩm nữa. Mất 1093.20 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 395 sản phẩm nữa. Mất 3593.47 mili giây
Đã thu thập và ghi 445 đánh giá của sản phẩm 21617915490 tại shop 788908334. Còn 394 sản phẩm nữa. Mất 22071.25 mili giây
Đã thu thập và ghi 6 đánh giá của sản phẩm 25400385770 tại shop 677686374. Còn 393 sản phẩm nữa. Mất 680.20 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 22485707071 tại shop 678666700. Còn 392 sản phẩm nữa. Mất 623.75 mili giây
Đã thu thập và ghi 525 đánh giá của sản phẩm 22658110026 tại shop 166361135. Còn 391 sản phẩm nữa. Mất 31629.12 mili giây
Đã thu thập và ghi 162 đánh giá của sản p

Đã thu thập và ghi 1 đánh giá của sản phẩm 19995035473 tại shop 678798230. Còn 329 sản phẩm nữa. Mất 624.88 mili giây
Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 328 sản phẩm nữa. Mất 188422.81 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 327 sản phẩm nữa. Mất 1041.32 mili giây
Đã thu thập và ghi 8 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 326 sản phẩm nữa. Mất 1110.15 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 325 sản phẩm nữa. Mất 3417.07 mili giây
Đã thu thập và ghi 445 đánh giá của sản phẩm 21617915490 tại shop 788908334. Còn 324 sản phẩm nữa. Mất 22205.64 mili giây
Đã thu thập và ghi 6 đánh giá của sản phẩm 25400385770 tại shop 677686374. Còn 323 sản phẩm nữa. Mất 617.20 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 22485707071 tại shop 678666700. Còn 322 sản phẩm nữa. Mất 595.54 mili giây
Đã thu thập và ghi 525 đánh giá của sản phẩm 

Đã thu thập và ghi 162 đánh giá của sản phẩm 23722617964 tại shop 950401869. Còn 260 sản phẩm nữa. Mất 7982.09 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 19995035473 tại shop 678798230. Còn 259 sản phẩm nữa. Mất 586.57 mili giây
Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 258 sản phẩm nữa. Mất 198683.07 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 257 sản phẩm nữa. Mất 1097.11 mili giây
Đã thu thập và ghi 8 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 256 sản phẩm nữa. Mất 902.14 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 255 sản phẩm nữa. Mất 3647.94 mili giây
Đã thu thập và ghi 445 đánh giá của sản phẩm 21617915490 tại shop 788908334. Còn 254 sản phẩm nữa. Mất 22285.62 mili giây
Đã thu thập và ghi 6 đánh giá của sản phẩm 25400385770 tại shop 677686374. Còn 253 sản phẩm nữa. Mất 605.50 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 

Đã thu thập và ghi 529 đánh giá của sản phẩm 22658110026 tại shop 166361135. Còn 191 sản phẩm nữa. Mất 33085.34 mili giây
Đã thu thập và ghi 162 đánh giá của sản phẩm 23722617964 tại shop 950401869. Còn 190 sản phẩm nữa. Mất 7580.30 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 19995035473 tại shop 678798230. Còn 189 sản phẩm nữa. Mất 550.66 mili giây
Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 188 sản phẩm nữa. Mất 200463.27 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 187 sản phẩm nữa. Mất 1162.31 mili giây
Đã thu thập và ghi 8 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 186 sản phẩm nữa. Mất 1187.51 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 185 sản phẩm nữa. Mất 3515.77 mili giây
Đã thu thập và ghi 445 đánh giá của sản phẩm 21617915490 tại shop 788908334. Còn 184 sản phẩm nữa. Mất 23100.47 mili giây
Đã thu thập và ghi 6 đánh giá của sản 

Đã thu thập và ghi 1 đánh giá của sản phẩm 22485707071 tại shop 678666700. Còn 122 sản phẩm nữa. Mất 619.96 mili giây
Đã thu thập và ghi 530 đánh giá của sản phẩm 22658110026 tại shop 166361135. Còn 121 sản phẩm nữa. Mất 33165.11 mili giây
Đã thu thập và ghi 162 đánh giá của sản phẩm 23722617964 tại shop 950401869. Còn 120 sản phẩm nữa. Mất 8041.55 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 19995035473 tại shop 678798230. Còn 119 sản phẩm nữa. Mất 560.21 mili giây
Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 118 sản phẩm nữa. Mất 182539.68 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 117 sản phẩm nữa. Mất 1123.89 mili giây
Đã thu thập và ghi 8 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 116 sản phẩm nữa. Mất 912.08 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 22967626489 tại shop 985237442. Còn 115 sản phẩm nữa. Mất 3408.34 mili giây
Đã thu thập và ghi 445 đánh giá của sản phẩ

Đã thu thập và ghi 6 đánh giá của sản phẩm 25400385770 tại shop 677686374. Còn 53 sản phẩm nữa. Mất 525.08 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 22485707071 tại shop 678666700. Còn 52 sản phẩm nữa. Mất 589.13 mili giây
Đã thu thập và ghi 530 đánh giá của sản phẩm 22658110026 tại shop 166361135. Còn 51 sản phẩm nữa. Mất 29499.08 mili giây
Đã thu thập và ghi 162 đánh giá của sản phẩm 23722617964 tại shop 950401869. Còn 50 sản phẩm nữa. Mất 7766.09 mili giây
Đã thu thập và ghi 1 đánh giá của sản phẩm 19995035473 tại shop 678798230. Còn 49 sản phẩm nữa. Mất 587.37 mili giây
Đã thu thập và ghi 3260 đánh giá của sản phẩm 3261115659 tại shop 769085. Còn 48 sản phẩm nữa. Mất 188025.35 mili giây
Đã thu thập và ghi 15 đánh giá của sản phẩm 20883341400 tại shop 533312313. Còn 47 sản phẩm nữa. Mất 1104.74 mili giây
Đã thu thập và ghi 8 đánh giá của sản phẩm 18583831164 tại shop 178569040. Còn 46 sản phẩm nữa. Mất 904.03 mili giây
Đã thu thập và ghi 58 đánh giá của sản phẩm 229676264

In [3]:
import pandas as pd

In [9]:
data = pd.read_csv('reviews_45s.csv')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210059 entries, 0 to 210058
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   userid       210059 non-null  int64 
 1   cmtid        210059 non-null  int64 
 2   untititle1   210059 non-null  int64 
 3   untitle2     210059 non-null  int64 
 4   mtime        210059 non-null  object
 5   rating_star  210059 non-null  int64 
 6   comment      210059 non-null  object
dtypes: int64(5), object(2)
memory usage: 11.2+ MB


In [11]:
data['comment']

0         Shop phục vụ quá tận tình, quay video chi tiết...
1         Đúng với mô tảok. Chất liệuok. Màu sắcok. Giao...
2         Khăn xinh lắm mn ơi e săn sale ở live nên là r...
3         Khăn len dày vừa phải, chất cũng ok. Màu đỏ đẹ...
4         Chất liệuvải mềm có lông lông. Đúng với mô tảđ...
                                ...                        
210054                                              xin nha
210055        Chất lượng tuyệt vời. Giá hợp lý. Đóng gói ổn
210056            Sản phẩm đúng mong muốn giá cả phải chăng
210057            Giá vừa phải nhưng dùng thì cũng tốt nha,
210058    Đã nhận được hàng. Đóng hộp đẹp Dùng được sẽủnghộ
Name: comment, Length: 210059, dtype: object